In [1]:
import numpy as np
import torch
from environment.Gridworld import Gridworld
from IPython.display import clear_output
import random
from matplotlib import pylab as plt
from collections import deque
from common.test import *
from environment.MarketEnv import MarketEnv

from agent.properties import *

if torch.cuda.is_available():
    devid = torch.device('cuda:0')
else:
    devid = torch.device('cpu')

for i in range(epochs):
    
    # numpy representation of initial state , reset
    # game = Gridworld(size=4, mode='random')

    marketEnv = MarketEnv()
    # rendered_game_boad_1 = game.board.render_np()
    state1_ = marketEnv.reset()
    # state1_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0

    state1 = torch.from_numpy(state1_).float()
    status = 1
    mov = 0
    while(status == 1): 
        j+=1
        mov += 1
        qval = model(state1)
        qval_ = qval.data.numpy()
        if (random.random() < epsilon):
            action_ = np.random.randint(0,4)
        else:
            action_ = np.argmax(qval_)
        
        action = action_
        
        # Execute action and upate state, and get reward + boolTerminal
        marketEnv.step(action)
        
        # rendered_game_boad_2 = game.board.render_np()
        state2_, reward, done, info_dic = marketEnv.step(action)
        # state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
        state2 = torch.from_numpy(state2_).float()

        exp =  (state1, action_, reward, state2, done)
        replay.append(exp) #H
        state1 = state2
        
        if len(replay) > batch_size:
            minibatch = random.sample(replay, batch_size)

            # Could be replaced with pytorch gather
            state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in minibatch]).view(batch_size, l1).to(device = devid)
            action_batch = torch.tensor([a for (s1,a,r,s2,d) in minibatch]).type(torch.FloatTensor).to(device = devid)
            reward_batch = torch.tensor([r for (s1,a,r,s2,d) in minibatch]).type(torch.FloatTensor).to(device = devid)
            state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in minibatch]).view(batch_size, l1).to(device = devid)
            done_batch = torch.tensor([d for (s1,a,r,s2,d) in minibatch]).type(torch.FloatTensor).to(device = devid)

            # Q update
            Q1 = model(state1_batch).to(device = devid)
            with torch.no_grad():
                Q2 = model2(state2_batch).to(device = devid) #B
            
            Y = reward_batch + gamma * ((1-done_batch) * torch.max(Q2,dim=1)[0])
            X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y.detach())
            print(i, loss.item())
            # clear_output(wait=True)
            optimizer.zero_grad()
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
            
            if j % sync_freq == 0: #C
                model2.load_state_dict(model.state_dict())
        if reward != -1 or mov > max_moves:
            status = 0
            mov = 0
        
losses = np.array(losses)

plt.figure(figsize=(10,7))
plt.plot(losses)
plt.xlabel("Epochs",fontsize=22)
plt.ylabel("Loss",fontsize=22)

3 350037.90625
4 241332.8125
5 243750.046875
6 119204.7265625
7 264064.625
8 281898.25
9 202596.015625
10 242275.078125
11 196046.984375
12 251834.171875
13 228921.25
14 152670.3125
15 300523.53125
16 46286.88671875
17 139371.703125
18 294401.03125
19 198518.703125
20 216635.703125
21 117673.5390625
22 238716.109375
23 304641.46875
24 152006.3125
25 122292.3828125
26 112185.1953125
27 176004.828125
28 136651.578125
29 167295.734375
30 238382.765625
31 151892.796875
32 82679.8515625
33 260934.203125
34 81966.9609375
35 55744.51953125
36 68741.6875
37 125649.1875
38 106598.1171875
39 71281.7734375
40 86733.8515625
41 193267.609375
42 53805.23828125
43 62636.86328125
44 194083.359375
45 133067.671875
46 8200.6875
47 13775.3388671875
48 42124.92578125
49 133578.734375
50 41140.34765625
51 17586.001953125
52 90655.1171875
53 62905.69140625
54 33769.45703125
55 26534.6640625
56 1416.7950439453125
57 67933.5
58 17961.392578125
59 18747.775390625
60 75996.359375
61 36129.98828125
62 34840.6562

KeyboardInterrupt: 